Loading model and packages

In [ ]:
!pip install yfinance
import yfinance as yf
from keras.models import load_model
from google.colab import  drive
drive.mount('/drive')
stock = "MSFT"
filename = "/drive/My Drive/Colab Notebooks/%sModel" % stock
loadModel = load_model(filename)

Mounted at /drive


Extracting data for forecasts of the current trading day

In [ ]:
import numpy as np
lastDate = "2022-08-18" 
numStepsDays = 10
newStockData = yf.download(stock,start='2022-07-03',progress=False,)
newStockData.reset_index(inplace=True)
lastDateIndx = np.where(newStockData.Date == lastDate)[0][0]
newStockData = newStockData[lastDateIndx-numStepsDays:]
newStockData = newStockData.reset_index(drop=True)

In [ ]:
newStockData.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
28,2022-09-14,253.529999,254.229996,249.860001,252.220001,252.220001,23913000
29,2022-09-15,249.770004,251.399994,244.020004,245.380005,245.380005,31530900
30,2022-09-16,244.259995,245.300003,242.059998,244.740005,244.740005,39775100
31,2022-09-19,242.470001,245.139999,240.850006,244.520004,244.520004,26826900
32,2022-09-20,242.070007,243.509995,239.639999,242.449997,242.449997,26606500


In [ ]:
#newStockData =newStockData.iloc[:len(newStockData)-1]

Data Preparation for current day forecasts

In [ ]:
#features = ["Close", "CloseChange", "VolChange","PriceChange", "Open", "High", "Low", "Adj Close", "Volume"]
features = ["Adj Close","Close","Open", "High", "Low"]#, "Volume"]
target = ["Adj Close"]

# create a differenced dataset
def differenceData(data, features, interval=1):
  diffData = data.copy()
  for var in features :
    diff = [np.nan]
    for i in range(interval, len(data)):
      value = diffData[var][i] - diffData[var][i - interval]
      diff.append(value)
    diffData[var] = diff
  return diffData

# invert differenced forecast
def inverseDifference(lastObs, value):
	return value + lastObs

def dataConversionForecast(data, numSteps):
    X= list()
    for i in range(len(data)):
        endIndx = i + numSteps
        if endIndx > len(data):
            break
        dataX = data[i:endIndx]
        X.append(dataX)
    return np.asarray(X).astype(np.float32)


In [ ]:
newDiffStockData = differenceData(newStockData,features)
newDiffStockData = newDiffStockData[1:].reset_index(drop = True)
newStockData = newStockData[1:].reset_index(drop = True)

from sklearn.preprocessing import StandardScaler
# train the normalization
scalerNew = StandardScaler()
scalerNew = scalerNew.fit(newDiffStockData[features])
# normalize the dataset and print
stdTestDataNew = scalerNew.transform(newDiffStockData[features])

lastDateIndxNew = np.where(newStockData.Date == lastDate)[0][0]
dataXTestNew = dataConversionForecast(stdTestDataNew, numStepsDays)
print(dataXTestNew.shape)
forecastNew = loadModel.predict(dataXTestNew)
forecastDummyNew = np.repeat(forecastNew,len(features), axis = -1)
predForecastsNew = scalerNew.inverse_transform(forecastDummyNew)[:,0] 

lastTargetTrained = newStockData[target].values[lastDateIndxNew][0]
predInvertedNew = list()
actualTargetsNew = newStockData[target][numStepsDays:].values
predInvertedNew.append(inverseDifference(lastTargetTrained, predForecastsNew[0]))
for i in range(1, len(predForecastsNew)):
  newPred = inverseDifference(actualTargetsNew[i-1], predForecastsNew[i])
  predInvertedNew.append(newPred[0])

(23, 10, 5)


Storing the predicted forecast price for the current day in a variable

In [ ]:
nextDaysTarget = predInvertedNew[len(predInvertedNew)-1]

In [ ]:
nextDaysTarget

241.66626346111298

In [ ]:
!pip install alpaca-trade-api

Instantiating Connection with Alpaca API 

In [ ]:
import alpaca_trade_api as tradeapi

# API Info for fetching data, portfolio, etc. from Alpaca
BASE_URL = "https://paper-api.alpaca.markets"
ALPACA_API_KEY = "PKKWFGQG2NUBWBCBY517"
ALPACA_SECRET_KEY = "vx0PuUdocSGeKaTej7lWhRecKfO71OAEKGeDJX3l"

# Instantiate REST API Connection
api = tradeapi.REST(key_id=ALPACA_API_KEY, secret_key=ALPACA_SECRET_KEY, 
                    base_url=BASE_URL, api_version='v2')

In [ ]:
# Fetch Account
account = api.get_account()

# Print Account Details
print(account.id, account.equity, account.status)

807d1a0b-e018-446a-9e69-f5147c7e0b16 100178.44 ACTIVE


In [ ]:
!pip install ta 
from ta.trend import ADXIndicator

Functions for Making and Deriving Trade Decisions

In [ ]:
def getClosingData(stock, lmt = 5000):
    # Returns a an numpy array of the closing prices of the past 5 minutes
    data = api.get_bars(symbol=stock,timeframe=tradeapi.TimeFrame.Minute, limit=lmt) 
    #data = newStockData
    closePrices5 = []
    for i in range(data.df.shape[0]-5,data.df.shape[0]):
        closePrices5.append(data[i].c)
    #for i in range(data.shape[0]-5,data.shape[0]):
    #    closePrices5.append(data["Close"][i])
    
    closePrices5 = np.array(closePrices5, dtype=np.float64)
    lastClosePrice = closePrices5[len(closePrices5)-1]
    closeMA5 = np.mean(closePrices5)

    data = data.df
    #from ta.trend import ADXIndicator
    ADX = ADXIndicator(data['high'],data['low'],data['close'],7)
    #ADX = ADXIndicator(data['High'],data['Low'],data['Close'],7)
    ADXValue = ADX.adx()
    posDI = ADX.adx_pos()
    negDI = ADX.adx_neg()
    #posLast = posDI[len(posDI)-1]
    #negLast = negDI[len(negDI)-1]
    #adxLast = ADXValue[len(ADXValue)-1]
    #if posDI[len(posDI)-1] > negDI[len(negDI)-1] and ADXValue[len(ADXValue)-1] > 25:

    return lastClosePrice, closeMA5, ADXValue, posDI, negDI


def buyStock(amount, stock): # Returns nothing, makes call to buy stock
    api.submit_order(symbol=stock, qty=amount, side='buy', 
                     type='market', time_in_force='day')
    print("Buying Stock")

def sellStock(amount, stock): # Returns nothing, makes call to sell stock
    api.submit_order(symbol=stock, qty=amount, side='sell', 
                     type='market', time_in_force='day')
    print("Selling Stock")

def buyTrailStop(amount, stock, perc): # Returns nothing, makes call to buy stock
    api.submit_order(symbol=stock, qty=amount, side='buy', 
                     type='market', time_in_force='day', trail_percent= perc)
    print("Buying Trail Stock")

def sellTrailStop(amount, stock, perc): # Returns nothing, makes call to sell stock
    api.submit_order(symbol=stock, qty=amount, side='sell', 
                     type='trailing_stop', time_in_force='day', trail_percent= perc)
    print("Selling Trail Stock")


Trading Strategy and Execution
Starts its exit strategies 3000 seconds before scheduled close of day

In [ ]:
import numpy as np
import time 
execute = True
stockHeld = False
while execute:
  stockClock = api.get_clock()
  nextCloseTime = stockClock.next_close
  if stockClock.is_open:
    lastClosePrice, closeMA5, ADXValue, posDI, negDI = getClosingData(stock, lmt = 5000)
    snapshot = api.get_snapshot(symbol=stock)
    latestTradePrice = snapshot.latest_trade.p
    previousLow = snapshot.prev_daily_bar.l
    previousHigh = snapshot.prev_daily_bar.h
    previousClose = snapshot.prev_daily_bar.c
    lenData = len(ADXValue)
            
    if lastClosePrice<nextDaysTarget:
      if (posDI[lenData-2] - negDI[lenData-2])<0 and (posDI[lenData-1] - negDI[lenData-1])>0:
        print("Price < nextDaysTarget and trend -ve to +ve",lastClosePrice) 
        buyStock(5,stock)
      #elif posDI[lenData-1] - negDI[lenData-1] > 0:
        #print("Price < nextDaysTarget and +ve trend")
        #buyStock(1,stock)
    if lastClosePrice>nextDaysTarget:
      if (posDI[lenData-2] - negDI[lenData-2])>0 and (posDI[lenData-1] - negDI[lenData-1])<0:
        try:
          sellStock(5,stock)
          print("Price > nextDaysTarget and trend +ve to -ve",lastClosePrice) 
        except:
          continue
    if ADXValue[lenData-1] > 25 and ADXValue[lenData-2]< 25 :
      if posDI[lenData-1] > negDI[lenData-1]:
        print("ADX Change : Buying",lastClosePrice) 
        buyStock(3,stock)
      elif posDI[lenData-1] < negDI[lenData-1]:
        try:
          sellStock(3,stock)
          print("ADX Change : Selling",lastClosePrice) 
        except:
          continue


    # Exit Strategy
    timetoClose = stockClock.next_close-stockClock.timestamp
    if timetoClose.seconds < 3000:
      positions = api.list_positions()
      for position in positions:
        if position.symbol==stock:
          currentPrice = np.float32(position.current_price)
          avgEntPrize = np.float32(position.avg_entry_price)
          margin = 1.001
          print(currentPrice,avgEntPrize,avgEntPrize*margin,timetoClose.seconds)
          qty = position.qty
          if int(qty) > 0 :
            if currentPrice>avgEntPrize*margin :
              sellStock(qty, stock)
              print("Closing all positions for a profit. Selling ",qty, " units of ", stock)
              execute = False
              if timetoClose.seconds < 2500:
                if currentPrice< avgEntPrize:
                  sellStock(int(qty), stock)
                  print("Closing all positions for a loss. Selling ",qty, " units of ", stock)
                  execute = False
          if int(qty) <0 :
            if currentPrice<avgEntPrize*margin :
              buyStock(abs(int(qty)), stock)
              print("Closing all positions. Buying ",abs(int(qty)), " units of ", stock)
              execute = False
              if timetoClose.seconds < 2500:
                if currentPrice > avgEntPrize:
                  buyStock(abs(int(qty)), stock)
                  print("Closing all positions for a loss. Buying ",qty, " units of ", stock)
                  execute = False


          
  time.sleep(60)